In [1]:
import os
os.environ['POLARS_MAX_THREADS'] = '16'
import pandas as pd
import polars as pl
from pathlib import Path

from esgpt_task_querying import main
from EventStream.data.dataset_polars import Dataset

pd.set_option('display.max_rows', 100)
pl.Config.set_tbl_cols(100)
pl.Config.set_tbl_rows(100)

data_path = '../MIMIC_ESD_new_schema_08-31-23-1'
os.getcwd()

/home/justinxu/miniconda3/envs/esgpt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'/home/justinxu/esgpt/ESGPTTaskQuerying/to_organize'

In [2]:
# DATA_DIR = Path(data_path)
# ESD = Dataset.load(DATA_DIR)

# events_df = ESD.events_df.filter(~pl.all_horizontal(pl.all().is_null()))
# dynamic_measurements_df = ESD.dynamic_measurements_df.filter(
#     ~pl.all_horizontal(pl.all().is_null())
# )

# ESD_data = (
#     events_df.join(dynamic_measurements_df, on="event_id", how="left")
#     .drop(["event_id"])
#     .sort(by=["subject_id", "timestamp", "event_type"])
# )

# if ESD_data["timestamp"].dtype != pl.Datetime:
#     ESD_data = ESD_data.with_columns(
#         pl.col("timestamp")
#         .str.strptime(pl.Datetime, format="%m/%d/%Y %H:%M")
#         .cast(pl.Datetime)
#     )

In [3]:
def has_event_type(type_str: str) -> pl.Expr:
    has_event_type = pl.col("event_type").cast(pl.Utf8).str.contains(type_str)
    # has_event_type = event_types.str.contains(type_str)
    return has_event_type

In [10]:
def validate_query(config_path, ESD_data, samples=10, seed=42):
    cfg = main.load_config(config_path)
    df_data = main.generate_predicate_columns(cfg, ESD_data)
    display(df_data)

    df_result = main.query_task(config_path, ESD_data)
    display(df_result)
    print(df_result.shape[0])

    if df_result.shape[0] == 0:
        print("No results found.")
        return

    try:
        validation = df_result.sample(samples, seed=seed)
        validation = validation.to_pandas()
    except:
        validation = df_result.to_pandas()

    # validation = df_result.filter(df_result['subject_id'] == 1916).to_pandas()

    for i, row in validation.iterrows():
        if i % 100 == 0:
            print(f"Validating row: {i}/{len(validation)}...")
        #filter ESD_data for the subject_id
        subject_id = row['subject_id']
        filtered_data = df_data.filter(pl.col("subject_id") == subject_id)

        print('Checking subject_id:', subject_id)

        #sort the values of every column that has 'timestamp' in its name
        timestamps = []
        for col in validation.columns:
            if 'timestamp' in col:
                timestamps.append((row[col], col))
        timestamps.sort(key=lambda x: x[0])

        filtered_data = filtered_data.filter(pl.col("timestamp") >= timestamps[0][0])
        filtered_data = filtered_data.filter(pl.col("timestamp") <= timestamps[-1][0])

        trigger_event = f'is_{cfg.windows.trigger.start}'
        assert filtered_data.filter(pl.col("timestamp") == row['trigger/timestamp']).select(trigger_event).to_pandas().values.flatten()[0] == 1
        
        after_trigger = False
        for i in range(len(timestamps)-1):
            window = timestamps[i]
            name = window[1].split('/')[0]
            print(f"Checking window: {name}->{timestamps[i+1][1].split('/')[0]}")
            if name == 'trigger':
                after_trigger = True
            if after_trigger:
                name = timestamps[i+1][1].split('/')[0]

            print(timestamps[i][0], timestamps[i+1][0])
            print(row[f'{name}/window_summary'])
            if cfg.windows[name].st_inclusive:
                window_data = filtered_data.filter(pl.col("timestamp") >= timestamps[i][0])
            else:
                window_data = filtered_data.filter(pl.col("timestamp") > timestamps[i][0])

            if cfg.windows[name].end_inclusive:
                window_data = window_data.filter(pl.col("timestamp") <= timestamps[i+1][0])
            else:
                window_data = window_data.filter(pl.col("timestamp") < timestamps[i+1][0])

            display(window_data)
            sum_counts = window_data.sum()
            display(sum_counts)

            for predicate in row[f'{name}/window_summary']:
                count = row[f'{name}/window_summary'][predicate]
                if not count:
                    count = 0
                assert sum_counts.select(predicate).to_pandas().values.flatten()[0] == count, (subject_id, timestamps[i][0], timestamps[i+1][0], row[f'{name}/window_summary'], sum_counts)

In [15]:
config_path = '../sample_configs/inhospital_mortality.yaml'
# config_path = '../sample_configs/abnormal_lab.yaml'
# config_path = '../sample_configs/imminent_mortality.yaml'
# config_path = '../sample_configs/intervention_weaning.yaml'
# config_path = '../sample_configs/long_term_incidence.yaml'
# config_path = '../sample_configs/readmission_risk.yaml'
df_temp = pl.from_pandas(pd.read_csv('test.csv'))
df_temp = df_temp.with_columns(
    pl.col("timestamp")
    .str.strptime(pl.Datetime, format="%m/%d/%Y %H:%M")
    .cast(pl.Datetime)
)
# display(df_temp)
validate_query(config_path, df_temp, samples=10, seed=42)

2024-04-14 01:14:43.673 | DEBUG    | esgpt_task_querying.event_predicates:generate_predicate_columns:118 - Added predicate column is_admission.
2024-04-14 01:14:43.674 | DEBUG    | esgpt_task_querying.event_predicates:generate_predicate_columns:118 - Added predicate column is_discharge.
2024-04-14 01:14:43.675 | DEBUG    | esgpt_task_querying.event_predicates:generate_predicate_columns:118 - Added predicate column is_death.
2024-04-14 01:14:43.676 | DEBUG    | esgpt_task_querying.event_predicates:generate_predicate_columns:118 - Added predicate column is_lab.
2024-04-14 01:14:43.677 | DEBUG    | esgpt_task_querying.event_predicates:generate_predicate_columns:125 - Added predicate column is_discharge_or_death.


shape: (1_056, 8)
┌────────────┬──────────────┬─────────────┬─────────────┬──────────┬─────────────┬────────┬────────┐
│ subject_id ┆ timestamp    ┆ is_admissio ┆ is_discharg ┆ is_death ┆ is_discharg ┆ is_lab ┆ is_any │
│ ---        ┆ ---          ┆ n           ┆ e           ┆ ---      ┆ e_or_death  ┆ ---    ┆ ---    │
│ i64        ┆ datetime[μs] ┆ ---         ┆ ---         ┆ i32      ┆ ---         ┆ i32    ┆ i32    │
│            ┆              ┆ i32         ┆ i32         ┆          ┆ i32         ┆        ┆        │
╞════════════╪══════════════╪═════════════╪═════════════╪══════════╪═════════════╪════════╪════════╡
│ 0          ┆ 2125-10-18   ┆ 1           ┆ 0           ┆ 0        ┆ 0           ┆ 0      ┆ 1      │
│            ┆ 16:50:00     ┆             ┆             ┆          ┆             ┆        ┆        │
│ 0          ┆ 2125-10-20   ┆ 0           ┆ 1           ┆ 0        ┆ 1           ┆ 0      ┆ 1      │
│            ┆ 15:15:00     ┆             ┆             ┆          ┆             ┆        ┆        │
│ 0          ┆ 2125-10-26   ┆ 1           ┆ 0           ┆ 0        ┆ 0           ┆ 0      ┆ 1      │
│            ┆ 20:32:00     ┆             ┆             ┆          ┆             ┆        ┆        │
│ 0          ┆ 2125-10-27   ┆ 0           ┆ 1           ┆ 0        ┆ 1           ┆ 0      ┆ 1      │
│            ┆ 16:28:00     ┆             ┆             ┆          ┆             ┆        ┆        │
│ 0          ┆ 2125-11-30   ┆ 1           ┆ 0           ┆ 0        ┆ 0           ┆ 0      ┆ 1      │
│            ┆ 16:02:00     ┆             ┆             ┆          ┆             ┆        ┆        │
│ 0          ┆ 2125-12-02   ┆ 0           ┆ 0           ┆ 0        ┆ 0           ┆ 1      ┆ 1      │
│            ┆ 16:02:00     ┆             ┆             ┆          ┆             ┆        ┆        │
│ 0          ┆ 2125-12-03   ┆ 0           ┆ 0           ┆ 0        ┆ 0           ┆ 1      ┆ 1      │
│            ┆ 14:43:00     ┆             ┆             ┆          ┆             ┆        ┆        │
│ 0          ┆ 2125-12-03   ┆ 0           ┆ 1           ┆ 0        ┆ 1           ┆ 0      ┆ 1      │
│            ┆ 14:44:00     ┆             ┆             ┆          ┆             ┆        ┆        │
│ 0          ┆ 2125-12-26   ┆ 1           ┆ 0           ┆ 0        ┆ 0           ┆ 0      ┆ 1      │
│            ┆ 17:06:00     ┆             ┆             ┆          ┆             ┆        ┆        │
│ 0          ┆ 2125-12-27   ┆ 0           ┆ 1           ┆ 0        ┆ 1           ┆ 0      ┆ 1      │
│            ┆ 16:52:00     ┆             ┆             ┆          ┆             ┆        ┆        │
│ 0          ┆ 2126-11-04   ┆ 1           ┆ 0           ┆ 0        ┆ 0           ┆ 0      ┆ 1      │
│            ┆ 00:50:00     ┆             ┆             ┆          ┆             ┆        ┆        │
│ 0          ┆ 2126-11-04   ┆ 0           ┆ 1           ┆ 0        ┆ 1           ┆ 0      ┆ 1      │
│            ┆ 12:49:00     ┆             ┆             ┆          ┆             ┆        ┆        │
│ 0          ┆ 2127-07-24   ┆ 1           ┆ 0           ┆ 0        ┆ 0           ┆ 0      ┆ 1      │
│            ┆ 18:06:00     ┆             ┆             ┆          ┆             ┆        ┆        │
│ 0          ┆ 2127-07-25   ┆ 0           ┆ 1           ┆ 0        ┆ 1           ┆ 0      ┆ 1      │
│            ┆ 17:41:00     ┆             ┆             ┆          ┆             ┆        ┆        │
│ 0          ┆ 2127-12-11   ┆ 1           ┆ 0           ┆ 0        ┆ 0           ┆ 0      ┆ 1      │
│            ┆ 18:06:00     ┆             ┆             ┆          ┆             ┆        ┆        │
│ 0          ┆ 2127-12-12   ┆ 0           ┆ 1           ┆ 0        ┆ 1           ┆ 0      ┆ 1      │
│            ┆ 13:46:00     ┆             ┆             ┆          ┆             ┆        ┆        │
│ 0          ┆ 2128-07-15   ┆ 1           ┆ 0           ┆ 0        ┆ 0           ┆ 0      ┆ 1      │
│            ┆ 18:44:00     ┆             ┆

2024-04-14 01:14:43.693 | DEBUG    | esgpt_task_querying.main:query_task:63 - Loading config...
2024-04-14 01:14:43.708 | DEBUG    | esgpt_task_querying.main:query_task:66 - Generating predicate columns...
2024-04-14 01:14:43.709 | DEBUG    | esgpt_task_querying.event_predicates:generate_predicate_columns:118 - Added predicate column is_admission.
2024-04-14 01:14:43.711 | DEBUG    | esgpt_task_querying.event_predicates:generate_predicate_columns:118 - Added predicate column is_discharge.
2024-04-14 01:14:43.711 | DEBUG    | esgpt_task_querying.event_predicates:generate_predicate_columns:118 - Added predicate column is_death.
2024-04-14 01:14:43.713 | DEBUG    | esgpt_task_querying.event_predicates:generate_predicate_columns:118 - Added predicate column is_lab.
2024-04-14 01:14:43.714 | DEBUG    | esgpt_task_querying.event_predicates:generate_predicate_columns:125 - Added predicate column is_discharge_or_death.
2024-04-14 01:14:43.717 | DEBUG    | esgpt_task_querying.main:query_task:75

trigger
┣━━ gap
┃   ┗━━ target
┗━━ input
False True


shape: (6, 9)
┌───────────┬───────────┬───────────┬──────────┬──────────┬──────────┬──────────┬──────────┬───────┐
│ subject_i ┆ trigger/t ┆ gap/times ┆ target/t ┆ input/ti ┆ gap/wind ┆ target/w ┆ input/wi ┆ label │
│ d         ┆ imestamp  ┆ tamp      ┆ imestamp ┆ mestamp  ┆ ow_summa ┆ indow_su ┆ ndow_sum ┆ ---   │
│ ---       ┆ ---       ┆ ---       ┆ ---      ┆ ---      ┆ ry       ┆ mmary    ┆ mary     ┆ i32   │
│ i64       ┆ datetime[ ┆ datetime[ ┆ datetime ┆ datetime ┆ ---      ┆ ---      ┆ ---      ┆       │
│           ┆ μs]       ┆ μs]       ┆ [μs]     ┆ [μs]     ┆ struct[6 ┆ struct[6 ┆ struct[6 ┆       │
│           ┆           ┆           ┆          ┆          ┆ ]        ┆ ]        ┆ ]        ┆       │
╞═══════════╪═══════════╪═══════════╪══════════╪══════════╪══════════╪══════════╪══════════╪═══════╡
│ 0         ┆ 2125-11-3 ┆ 2125-12-0 ┆ 2125-12- ┆ 2125-10- ┆ {0,0,0,0 ┆ {0,1,0,1 ┆ {1,0,0,0 ┆ 0     │
│           ┆ 0         ┆ 2         ┆ 03       ┆ 31       ┆ ,1,1}    ┆ ,1,2}    ┆ ,0,1}    ┆       │
│           ┆ 16:02:00  ┆ 16:02:00  ┆ 14:44:00 ┆ 16:02:00 ┆          ┆          ┆          ┆       │
│ 0         ┆ 2130-04-1 ┆ 2130-04-1 ┆ 2130-04- ┆ 2130-03- ┆ {0,0,0,0 ┆ {0,1,0,1 ┆ {2,1,0,1 ┆ 0     │
│           ┆ 5         ┆ 7         ┆ 19       ┆ 16       ┆ ,0,0}    ┆ ,0,1}    ┆ ,0,3}    ┆       │
│           ┆ 21:10:00  ┆ 21:10:00  ┆ 16:00:00 ┆ 21:10:00 ┆          ┆          ┆          ┆       │
│ 0         ┆ 2130-08-2 ┆ 2130-08-2 ┆ 2130-08- ┆ 2130-07- ┆ {0,0,0,0 ┆ {0,1,0,1 ┆ {1,0,0,0 ┆ 0     │
│           ┆ 1         ┆ 3         ┆ 23       ┆ 22       ┆ ,0,0}    ┆ ,0,1}    ┆ ,0,1}    ┆       │
│           ┆ 15:26:00  ┆ 15:26:00  ┆ 16:40:00 ┆ 15:26:00 ┆          ┆          ┆          ┆       │
│ 0         ┆ 2130-10-0 ┆ 2130-10-1 ┆ 2130-10- ┆ 2130-09- ┆ {0,0,0,0 ┆ {0,1,0,1 ┆ {2,1,0,1 ┆ 0     │
│           ┆ 8         ┆ 0         ┆ 12       ┆ 08       ┆ ,0,0}    ┆ ,0,1}    ┆ ,0,3}    ┆       │
│           ┆ 19:03:00  ┆ 19:03:00  ┆ 19:05:00 ┆ 19:03:00 ┆          ┆          ┆          ┆       │
│ 0         ┆ 2130-10-1 ┆ 2130-10-2 ┆ 2130-10- ┆ 2130-09- ┆ {0,0,0,0 ┆ {0,1,0,1 ┆ {4,3,0,3 ┆ 0     │
│           ┆ 9         ┆ 1         ┆ 22       ┆ 19       ┆ ,0,0}    ┆ ,1,2}    ┆ ,0,7}    ┆       │
│           ┆ 16:20:00  ┆ 16:20:00  ┆ 15:13:00 ┆ 16:20:00 ┆          ┆          ┆          ┆       │
│ 0         ┆ 2130-12-2 ┆ 2130-12-2 ┆ 2130-12- ┆ 2130-11- ┆ {0,0,0,0 ┆ {0,1,0,1 ┆ {4,3,0,3 ┆ 0     │
│           ┆ 7         ┆ 9         ┆ 30       ┆ 27       ┆ ,0,0}    ┆ ,0,1}    ┆ ,0,7}    ┆       │
│           ┆ 21:23:00  ┆ 21:23:00  ┆ 15:33:00 ┆ 21:23:00 ┆          ┆          ┆          ┆       │
└───────────┴───────────┴───────────┴──────────┴──────────┴──────────┴──────────┴──────────┴───────┘

...

shape: (0, 8)
┌────────────┬──────────────┬─────────────┬─────────────┬──────────┬─────────────┬────────┬────────┐
│ subject_id ┆ timestamp    ┆ is_admissio ┆ is_discharg ┆ is_death ┆ is_discharg ┆ is_lab ┆ is_any │
│ ---        ┆ ---          ┆ n           ┆ e           ┆ ---      ┆ e_or_death  ┆ ---    ┆ ---    │
│ i64        ┆ datetime[μs] ┆ ---         ┆ ---         ┆ i32      ┆ ---         ┆ i32    ┆ i32    │
│            ┆              ┆ i32         ┆ i32         ┆          ┆ i32         ┆        ┆        │
╞════════════╪══════════════╪═════════════╪═════════════╪══════════╪═════════════╪════════╪════════╡
└────────────┴──────────────┴─────────────┴─────────────┴──────────┴─────────────┴────────┴────────┘

shape: (1, 8)
┌────────────┬──────────────┬─────────────┬─────────────┬──────────┬─────────────┬────────┬────────┐
│ subject_id ┆ timestamp    ┆ is_admissio ┆ is_discharg ┆ is_death ┆ is_discharg ┆ is_lab ┆ is_any │
│ ---        ┆ ---          ┆ n           ┆ e           ┆ ---      ┆ e_or_death  ┆ ---    ┆ ---    │
│ i64        ┆ datetime[μs] ┆ ---         ┆ ---         ┆ i32      ┆ ---         ┆ i32    ┆ i32    │
│            ┆              ┆ i32         ┆ i32         ┆          ┆ i32         ┆        ┆        │
╞════════════╪══════════════╪═════════════╪═════════════╪══════════╪═════════════╪════════╪════════╡
│ 0          ┆ null         ┆ 0           ┆ 0           ┆ 0        ┆ 0           ┆ 0      ┆ 0      │
└────────────┴──────────────┴─────────────┴─────────────┴──────────┴─────────────┴────────┴────────┘

Checking window: gap->target
2130-12-29 21:23:00 2130-12-30 15:33:00
{'is_admission': 0, 'is_discharge': 1, 'is_death': 0, 'is_discharge_or_death': 1, 'is_lab': 0, 'is_any': 1}


shape: (1, 8)
┌────────────┬──────────────┬─────────────┬─────────────┬──────────┬─────────────┬────────┬────────┐
│ subject_id ┆ timestamp    ┆ is_admissio ┆ is_discharg ┆ is_death ┆ is_discharg ┆ is_lab ┆ is_any │
│ ---        ┆ ---          ┆ n           ┆ e           ┆ ---      ┆ e_or_death  ┆ ---    ┆ ---    │
│ i64        ┆ datetime[μs] ┆ ---         ┆ ---         ┆ i32      ┆ ---         ┆ i32    ┆ i32    │
│            ┆              ┆ i32         ┆ i32         ┆          ┆ i32         ┆        ┆        │
╞════════════╪══════════════╪═════════════╪═════════════╪══════════╪═════════════╪════════╪════════╡
│ 0          ┆ 2130-12-30   ┆ 0           ┆ 1           ┆ 0        ┆ 1           ┆ 0      ┆ 1      │
│            ┆ 15:33:00     ┆             ┆             ┆          ┆             ┆        ┆        │
└────────────┴──────────────┴─────────────┴─────────────┴──────────┴─────────────┴────────┴────────┘

shape: (1, 8)
┌────────────┬──────────────┬─────────────┬─────────────┬──────────┬─────────────┬────────┬────────┐
│ subject_id ┆ timestamp    ┆ is_admissio ┆ is_discharg ┆ is_death ┆ is_discharg ┆ is_lab ┆ is_any │
│ ---        ┆ ---          ┆ n           ┆ e           ┆ ---      ┆ e_or_death  ┆ ---    ┆ ---    │
│ i64        ┆ datetime[μs] ┆ ---         ┆ ---         ┆ i32      ┆ ---         ┆ i32    ┆ i32    │
│            ┆              ┆ i32         ┆ i32         ┆          ┆ i32         ┆        ┆        │
╞════════════╪══════════════╪═════════════╪═════════════╪══════════╪═════════════╪════════╪════════╡
│ 0          ┆ null         ┆ 0           ┆ 1           ┆ 0        ┆ 1           ┆ 0      ┆ 1      │
└────────────┴──────────────┴─────────────┴─────────────┴──────────┴─────────────┴────────┴────────┘

In [6]:
config_path = '../sample_configs/inhospital_mortality.yaml'
df_result = main.query_task(config_path, ESD_data, verbose=True)

Loading config...

Generating predicate columns...



Added predicate column is_admission.
Added predicate column is_discharge.
Added predicate column is_death.
Added predicate column is_discharge_or_death.

Building tree...
trigger
┣━━ gap
┃   ┗━━ target
┗━━ input


12127 subjects (14623763 rows) were excluded due to trigger event: admission.


Querying...


Querying subtree rooted at gap...
342 subjects (631 rows) were excluded due to constraint: [(col("is_admission")) <= (0)].
7143 subjects (17534 rows) were excluded due to constraint: [(col("is_discharge")) <= (0)].


Querying subtree rooted at target...
75 subjects (100 rows) were excluded due to constraint: [(col("is_discharge_or_death")) >= (1)].
5721 subjects (14185 rows) were excluded due to constraint: [(col("is_discharge_or_death")) <= (1)].


Querying subtree rooted at input...


Done.



In [7]:
df_result

shape: (52_418, 9)
┌───────────┬───────────┬───────────┬──────────┬──────────┬──────────┬──────────┬──────────┬───────┐
│ subject_i ┆ trigger/t ┆ gap/times ┆ target/t ┆ input/ti ┆ gap/wind ┆ target/w ┆ input/wi ┆ label │
│ d         ┆ imestamp  ┆ tamp      ┆ imestamp ┆ mestamp  ┆ ow_summa ┆ indow_su ┆ ndow_sum ┆ ---   │
│ ---       ┆ ---       ┆ ---       ┆ ---      ┆ ---      ┆ ry       ┆ mmary    ┆ mary     ┆ i32   │
│ u16       ┆ datetime[ ┆ datetime[ ┆ datetime ┆ datetime ┆ ---      ┆ ---      ┆ ---      ┆       │
│           ┆ μs]       ┆ μs]       ┆ [μs]     ┆ [μs]     ┆ struct[5 ┆ struct[5 ┆ struct[5 ┆       │
│           ┆           ┆           ┆          ┆          ┆ ]        ┆ ]        ┆ ]        ┆       │
╞═══════════╪═══════════╪═══════════╪══════════╪══════════╪══════════╪══════════╪══════════╪═══════╡
│ 0         ┆ 2125-11-3 ┆ 2125-12-0 ┆ 2125-12- ┆ 1852-02- ┆ {0,0,0,0 ┆ {0,1,0,1 ┆ {3,2,0,2 ┆ 0     │
│           ┆ 0         ┆ 2         ┆ 03       ┆ 16       ┆ ,0}      ┆ ,1}      ┆ ,5}      ┆       │
│           ┆ 16:02:00  ┆ 16:02:00  ┆ 14:44:00 ┆ 16:02:00 ┆          ┆          ┆          ┆       │
│ 0         ┆ 2130-04-1 ┆ 2130-04-1 ┆ 2130-04- ┆ 1856-07- ┆ {0,0,0,0 ┆ {0,1,0,1 ┆ {10,9,0, ┆ 0     │
│           ┆ 5         ┆ 7         ┆ 19       ┆ 01       ┆ ,0}      ┆ ,1}      ┆ 9,19}    ┆       │
│           ┆ 21:10:00  ┆ 21:10:00  ┆ 16:00:00 ┆ 21:10:00 ┆          ┆          ┆          ┆       │
│ 0         ┆ 2130-08-2 ┆ 2130-08-2 ┆ 2130-08- ┆ 1856-11- ┆ {0,0,0,0 ┆ {0,1,0,1 ┆ {12,11,0 ┆ 0     │
│           ┆ 1         ┆ 3         ┆ 23       ┆ 06       ┆ ,0}      ┆ ,1}      ┆ ,11,23}  ┆       │
│           ┆ 15:26:00  ┆ 15:26:00  ┆ 16:40:00 ┆ 15:26:00 ┆          ┆          ┆          ┆       │
│ 0         ┆ 2130-10-0 ┆ 2130-10-1 ┆ 2130-10- ┆ 1856-12- ┆ {0,0,0,0 ┆ {0,1,0,1 ┆ {14,13,0 ┆ 0     │
│           ┆ 8         ┆ 0         ┆ 12       ┆ 24       ┆ ,0}      ┆ ,1}      ┆ ,13,27}  ┆       │
│           ┆ 19:03:00  ┆ 19:03:00  ┆ 19:05:00 ┆ 19:03:00 ┆          ┆          ┆          ┆       │
│ 0         ┆ 2130-10-1 ┆ 2130-10-2 ┆ 2130-10- ┆ 1857-01- ┆ {0,0,0,0 ┆ {0,1,0,1 ┆ {16,15,0 ┆ 0     │
│           ┆ 9         ┆ 1         ┆ 22       ┆ 04       ┆ ,0}      ┆ ,1}      ┆ ,15,31}  ┆       │
│           ┆ 16:20:00  ┆ 16:20:00  ┆ 15:13:00 ┆ 16:20:00 ┆          ┆          ┆          ┆       │
│ 0         ┆ 2130-12-2 ┆ 2130-12-2 ┆ 2130-12- ┆ 1857-03- ┆ {0,0,0,0 ┆ {0,1,0,1 ┆ {21,20,0 ┆ 0     │
│           ┆ 7         ┆ 9         ┆ 30       ┆ 14       ┆ ,0}      ┆ ,1}      ┆ ,20,41}  ┆       │
│           ┆ 21:23:00  ┆ 21:23:00  ┆ 15:33:00 ┆ 21:23:00 ┆          ┆          ┆          ┆       │
│ 0         ┆ 2131-01-0 ┆ 2131-01-0 ┆ 2131-01- ┆ 1857-03- ┆ {0,0,0,0 ┆ {0,1,1,1 ┆ {22,21,0 ┆ 1     │
│           ┆ 7         ┆ 9         ┆ 20       ┆ 25       ┆ ,0}      ┆ ,1009}   ┆ ,21,43}  ┆       │
│           ┆ 20:39:00  ┆ 20:39:00  ┆ 05:15:00 ┆ 20:39:00 ┆          ┆          ┆          ┆       │
│ 1         ┆ 2128-07-2 ┆ 2128-07-3 ┆ 2128-07- ┆ 1854-10- ┆ {0,0,0,0 ┆ {0,1,0,1 ┆ {1,0,0,0 ┆ 0     │
│           ┆ 9         ┆ 1         ┆ 31       ┆ 15       ┆ ,0}      ┆ ,1}      ┆ ,1}      ┆       │
│           ┆ 17:01:00  ┆ 17:01:00  ┆ 18:00:00 ┆ 17:01:00 ┆          ┆          ┆          ┆       │
│ 1         ┆ 2129-08-0 ┆ 2129-08-0 ┆ 2129-08- ┆ 1855-10- ┆ {0,0,0,0 ┆ {0,1,0,1 ┆ {2,1,0,1 ┆ 0     │
│           ┆ 4         ┆ 6         ┆ 18       ┆ 21       ┆ ,124}    ┆ ,242}    ┆ ,3}      ┆       │
│           ┆ 12:44:00  ┆ 12:44:00  ┆ 16:53:00 ┆ 12:44:00 ┆          ┆          ┆          ┆       │
│ 1         ┆ 2130-09-2 ┆ 2130-09-2 ┆ 2130-09- ┆ 1856-12- ┆ {0,0,0,0 ┆ {0,1,0,1 ┆ {4,3,0,3 ┆ 0     │
│           ┆ 3         ┆ 5         ┆ 29       ┆ 09       ┆ ,113}    ┆ ,101}    ┆ ,372}    ┆       │
│           ┆ 21:59:00  ┆ 21:59:00  ┆ 18:55:00 ┆ 21:59:00 ┆          ┆          ┆          ┆       │
│ 2         ┆ 2156-04-1 ┆ 2156-04-1 ┆ 2156-04- ┆ 1882-06- ┆ {0,0,0,0 ┆ {0,1,0,1 ┆ {2,1,0,1 ┆ 0     │
│           ┆ 2         ┆ 4         ┆ 29  